In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
import xgboost as xgb
from datetime import datetime, timedelta


In [52]:
# Load all datasets
sales = pd.read_csv('../../data/processed/sales_processed.csv')
products = pd.read_csv('../../data/processed/products_processed.csv')
cities = pd.read_csv('../../data/processed/city_processed.csv')
competitor = pd.read_csv('../../data/processed/competitors_processed.csv')
market_influence = pd.read_csv('../../data/processed/market_influencers_processed.csv')
customers = pd.read_csv('../../data/processed/consumer_behavior_processed.csv')

In [53]:

# Convert dates to datetime
sales['Date'] = pd.to_datetime(sales['Date'])
competitor['Date'] = pd.to_datetime(competitor['Date'])
market_influence['Week_Start_Date'] = pd.to_datetime(market_influence['Week_Start_Date'])

In [ ]:
# Merge all datasets
# Base sales data with products
merged = sales.merge(
    products.rename(columns={'SKU Identification Number': 'SKU_ID'}),
    on='SKU_ID'
)

# Add city data
merged = merged.merge(cities, on='City_ID')



In [ ]:


def align_and_fill_weekly(daily_df, weekly_df, group_cols=['City_ID']):
    """Align weekly data with forward-fill imputation"""
    # Merge with weekly data
    merged = pd.merge_asof(
        daily_df.sort_values('Date'),
        weekly_df.sort_values('Date'),
        on='Date',
        by=group_cols,
        direction='nearest',
        tolerance=pd.Timedelta('7D')
    )
    
    # Forward fill weekly values within groups
    weekly_cols = list(weekly_df.columns.difference(daily_df.columns))
    merged[weekly_cols] = merged.groupby(group_cols)[weekly_cols].ffill()
    
    return merged
market_influence.rename(columns={'Week_Start_Date': 'Date'}, inplace=True)

merged = align_and_fill_weekly(
    merged,
    market_influence.groupby(['City_ID', pd.Grouper(key='Date', freq='W-MON')])
    .agg({'Avg_Temperature': 'mean', 'Weather_Type': lambda x: x.mode()[0], 'Festival': 'first'})
    .reset_index(),
    group_cols=['City_ID']
)

merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Product Name,Flavor Variant,Launch Date,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce,City_Name,City_tier,Population_Density(persons/km),Per_Capita_Income (INR),Avg_Temperature,Weather_Type,Festival
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival
1,2023-01-01,CT014,SKU1007,General Trade,428,32100.0,Minute Maid 250ml Pulpy Orange,Orange,2015-04-23,0.25,1,1,1,1,0,Chandigarh,Tier 2,9250,349000,19.7,Cold,No Festival
2,2023-01-01,CT014,SKU1002,General Trade,1125,337.5,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Chandigarh,Tier 2,9250,349000,19.7,Cold,No Festival
3,2023-01-01,CT014,SKU1001,General Trade,938,281.4,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Chandigarh,Tier 2,9250,349000,19.7,Cold,No Festival
4,2023-01-01,CT014,SKU1008,Modern Trade,207,15525.0,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,2015-04-23,0.25,1,1,1,1,0,Chandigarh,Tier 2,9250,349000,19.7,Cold,No Festival
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT018,SKU1007,General Trade,288,21600.0,Minute Maid 250ml Pulpy Orange,Orange,2015-04-23,0.25,1,1,1,1,0,Patna,Tier 2,2805,173305,19.8,Cold,No Festival
996,2023-01-03,CT016,SKU1002,Q Commerce,187,56.1,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Nagpur,Tier 2,9664,296607,21.3,Cold,No Festival
997,2023-01-03,CT018,SKU1002,General Trade,288,86.4,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Patna,Tier 2,2805,173305,19.8,Cold,No Festival
998,2023-01-03,CT018,SKU1008,Modern Trade,92,6900.0,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,2015-04-23,0.25,1,1,1,1,0,Patna,Tier 2,2805,173305,19.8,Cold,No Festival


In [ ]:


merged = align_and_fill_weekly(
    merged,
    competitor.groupby(['Channel', pd.Grouper(key='Date', freq='W-MON')])
    .agg({'Mentions_Count': 'sum', 'Sentiment_Score': 'mean', 'Share_of_Voice': 'mean'})
    .reset_index(),
    group_cols=['Channel']
)

merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Product Name,Flavor Variant,Launch Date,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce,City_Name,City_tier,Population_Density(persons/km),Per_Capita_Income (INR),Avg_Temperature,Weather_Type,Festival,Mentions_Count,Sentiment_Score,Share_of_Voice
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,531,69.000000,50.000000
1,2023-01-01,CT010,SKU1008,HoReCa,8,600.0,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,2015-04-23,0.25,1,1,1,1,0,Lucknow,Tier 2,3814,105000,18.4,Cold,No Festival,543,67.666667,33.333333
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0,Minute Maid 250ml Pulpy Orange,Orange,2015-04-23,0.25,1,1,1,1,0,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT007,SKU1001,E Commerce,229,68.7,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Ahmedabad,Tier 1,12984,302588,21.0,Cold,No Festival,1044,67.400000,33.366667
996,2023-01-03,CT003,SKU1004,Modern Trade,95,28.5,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Kolkata,Tier 1,24252,171184,19.8,Cold,No Festival,884,73.400000,50.000000
997,2023-01-03,CT006,SKU1004,Q Commerce,85,25.5,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Hyderabad,Tier 1,10477,400000,18.8,Cold,No Festival,531,69.000000,50.000000
998,2023-01-03,CT008,SKU1004,Modern Trade,209,62.7,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Pune,Tier 1,6034,336503,20.4,Cold,No Festival,884,73.400000,50.000000


In [57]:

# Add customer preferences
customer_agg = customers.groupby('City_ID').agg({
    'Preferred_Channel': lambda x: x.mode()[0],
    'Income_Level': 'mean',
    'Price_Sensitivity': lambda x: x.mode()[0],
    'Purchase_Frequency': 'mean'
}).reset_index().add_prefix('Customer_')

merged = merged.merge(
    customer_agg.rename(columns={'Customer_City_ID': 'City_ID'}),
    on='City_ID',
    how='left'
)

merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Product Name,Flavor Variant,Launch Date,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce,City_Name,City_tier,Population_Density(persons/km),Per_Capita_Income (INR),Avg_Temperature,Weather_Type,Festival,Mentions_Count,Sentiment_Score,Share_of_Voice,Customer_Preferred_Channel,Customer_Income_Level,Customer_Price_Sensitivity,Customer_Purchase_Frequency
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,531,69.000000,50.000000,E-commerce,520661.105099,Low,3.111607
1,2023-01-01,CT010,SKU1008,HoReCa,8,600.0,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,2015-04-23,0.25,1,1,1,1,0,Lucknow,Tier 2,3814,105000,18.4,Cold,No Festival,543,67.666667,33.333333,E-commerce,120715.451706,High,3.057965
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,520661.105099,Low,3.111607
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,520661.105099,Low,3.111607
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0,Minute Maid 250ml Pulpy Orange,Orange,2015-04-23,0.25,1,1,1,1,0,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,520661.105099,Low,3.111607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT007,SKU1001,E Commerce,229,68.7,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Ahmedabad,Tier 1,12984,302588,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,341307.248214,High,3.062574
996,2023-01-03,CT003,SKU1004,Modern Trade,95,28.5,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Kolkata,Tier 1,24252,171184,19.8,Cold,No Festival,884,73.400000,50.000000,E-commerce,194478.497198,High,3.026316
997,2023-01-03,CT006,SKU1004,Q Commerce,85,25.5,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Hyderabad,Tier 1,10477,400000,18.8,Cold,No Festival,531,69.000000,50.000000,E-commerce,457865.997231,Low,3.069527
998,2023-01-03,CT008,SKU1004,Modern Trade,209,62.7,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Pune,Tier 1,6034,336503,20.4,Cold,No Festival,884,73.400000,50.000000,E-commerce,379895.103012,High,3.033946


In [58]:
merged['Days_Since_Launch'] = (
    merged['Date'] - pd.to_datetime(merged['Launch Date'])
).dt.days

# Temporal features
merged['Day_of_week'] = merged['Date'].dt.dayofweek
merged['Month'] = merged['Date'].dt.month
merged['Is_month_end'] = merged['Date'].dt.is_month_end.astype(int)

merged.head(1000)

,Date,City_ID,SKU_ID,Channel,Units_Sold,Sales,Product Name,Flavor Variant,Launch Date,Pack Size (L),General Trade,E Commerce,Modern Trade,HoReCa,Q Commerce,City_Name,City_tier,Population_Density(persons/km),Per_Capita_Income (INR),Avg_Temperature,Weather_Type,Festival,Mentions_Count,Sentiment_Score,Share_of_Voice,Customer_Preferred_Channel,Customer_Income_Level,Customer_Price_Sensitivity,Customer_Purchase_Frequency,Days_Since_Launch,Day_of_week,Month,Is_month_end
0,2023-01-01,CT001,SKU1002,Q Commerce,268,80.4,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,531,69.000000,50.000000,E-commerce,520661.105099,Low,3.111607,2810,6,1,0
1,2023-01-01,CT010,SKU1008,HoReCa,8,600.0,Minute Maid 250ml Mixed Fruit Juice,Mixed Fruit,2015-04-23,0.25,1,1,1,1,0,Lucknow,Tier 2,3814,105000,18.4,Cold,No Festival,543,67.666667,33.333333,E-commerce,120715.451706,High,3.057965,2810,6,1,0
2,2023-01-01,CT001,SKU1001,E Commerce,521,156.3,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,520661.105099,Low,3.111607,316,6,1,0
3,2023-01-01,CT001,SKU1002,E Commerce,247,74.1,Minute Maid Mixed Fruit Juice,Mixed Fruit,2015-04-23,1.00,1,1,1,1,1,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,520661.105099,Low,3.111607,2810,6,1,0
4,2023-01-01,CT001,SKU1007,E Commerce,161,12075.0,Minute Maid 250ml Pulpy Orange,Orange,2015-04-23,0.25,1,1,1,1,0,Delhi,Tier 1,14893,461910,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,520661.105099,Low,3.111607,2810,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-03,CT007,SKU1001,E Commerce,229,68.7,Minute Maid Apple Juice - Honey Infused,Apple,2022-02-19,1.00,1,1,0,0,0,Ahmedabad,Tier 1,12984,302588,21.0,Cold,No Festival,1044,67.400000,33.366667,E-commerce,341307.248214,High,3.062574,318,1,1,0
996,2023-01-03,CT003,SKU1004,Modern Trade,95,28.5,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Kolkata,Tier 1,24252,171184,19.8,Cold,No Festival,884,73.400000,50.000000,E-commerce,194478.497198,High,3.026316,2812,1,1,0
997,2023-01-03,CT006,SKU1004,Q Commerce,85,25.5,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Hyderabad,Tier 1,10477,400000,18.8,Cold,No Festival,531,69.000000,50.000000,E-commerce,457865.997231,Low,3.069527,2812,1,1,0
998,2023-01-03,CT008,SKU1004,Modern Trade,209,62.7,Minute Maid Pulpy Orange,Orange,2015-04-23,1.00,1,1,1,1,1,Pune,Tier 1,6034,336503,20.4,Cold,No Festival,884,73.400000,50.000000,E-commerce,379895.103012,High,3.033946,2812,1,1,0


In [59]:

# Define features and target
target = 'Sales'
features = [
    'Channel', 'Flavor Variant', 'Pack Size (L)', 'City_tier',
    'Population_Density(persons/km)', 'Per_Capita_Income (INR)',
    'Avg_Temperature', 'Weather_Type', 'Festival',
    'Mentions_Count', 'Sentiment_Score', 'Share_of_Voice',
    'Customer_Preferred_Channel', 'Customer_Income_Level',
    'Customer_Price_Sensitivity', 'Days_Since_Launch',
    'Day_of_week', 'Month', 'Is_month_end'
]

# Split data
X = merged[features]
y = merged[target]

# Preprocessing pipeline
numeric_features = [
    'Pack Size (L)', 'Population_Density(persons/km)',
    'Per_Capita_Income (INR)', 'Avg_Temperature',
    'Mentions_Count', 'Sentiment_Score', 'Share_of_Voice',
    'Customer_Income_Level', 'Days_Since_Launch'
]
categorical_features = [
    'Channel', 'Flavor Variant', 'City_tier',
    'Weather_Type', 'Festival', 'Customer_Preferred_Channel',
    'Customer_Price_Sensitivity'
]
temporal_features = ['Day_of_week', 'Month', 'Is_month_end']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('temp', 'passthrough', temporal_features)
    ])

# 4. Temporal Validation
# Sort by date first
merged = merged.sort_values('Date')
X = merged[features]
y = merged[target]

# Time-based split (80-20)
split_date = merged['Date'].quantile(0.8)
X_train = X[merged['Date'] <= split_date]
y_train = y[merged['Date'] <= split_date]
X_test = X[merged['Date'] > split_date]
y_test = y[merged['Date'] > split_date]

# 5. Model Training
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=5,
        random_state=42
    ))
])

model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Pack Size (L)',
                                                   'Population_Density(persons/km)',
                                                   'Per_Capita_Income (INR)',
                                                   'Avg_Temperature',
                                                   'Mentions_Count',
                                                   'Sentiment_Score',
                                                   'Share_of_Voice',
                                                   'Customer_Income_Level',
                                                   'Days_Since_Launch']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Channel', 'Flavor Variant',
                                                   'City_tier', 'Weather_Type',
                                                   'Festival',
                                                   'Customer_Preferred_Channel',
                                                   'Customer_Price_Sensitivity']),
                                                 ('temp', 'passthrough',
                                                  ['Day_of_week', 'Month',
                                                   'Is_month_end'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=0.05, max_depth=5,
                                           n_estimators=200,
                                           random_state=42))])

In [60]:
def predict_optimal_channel(model, input_data):
    """Predict sales for all channels and return top channel"""
    # Generate predictions for all channels
    channels = ['General Trade', 'Modern Trade', 'E Commerce', 'HoReCa', 'Q Commerce']
    predictions = []
    
    for channel in channels:
        channel_data = input_data.copy()
        channel_data['Channel'] = channel
        pred = model.predict(channel_data)
        predictions.append(pred)
    
    # Find best channel
    best_idx = np.argmax(predictions)
    return channels[best_idx], predictions[best_idx]

# 7. Evaluation
# Basic regression metrics
y_pred = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

# Top-1 Accuracy (How often we predict the actual best channel)
test_with_pred = merged[merged['Date'] > split_date].copy()
test_with_pred['Predicted_Sales'] = y_pred

# Get actual best channel for each group
actual_best = test_with_pred.groupby(['Date', 'City_ID', 'SKU_ID'])['Sales']\
    .idxmax().apply(lambda x: test_with_pred.loc[x, 'Channel'])

# Get predicted best channel
predicted_best = test_with_pred.groupby(['Date', 'City_ID', 'SKU_ID'])\
    .apply(lambda x: x.loc[x['Predicted_Sales'].idxmax(), 'Channel'])

top1_accuracy = (actual_best == predicted_best).mean()
print(f"Top-1 Accuracy: {top1_accuracy:.2%}")


MAE: 2004.893711637322
RMSE: 4628.3020960709355
Top-1 Accuracy: 69.93%


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_6464\3007029064.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.loc[x['Predicted_Sales'].idxmax(), 'Channel'])


In [67]:
# 8. Example Usage for New Product
# Prepare new product data
new_product = {
    'Flavor Variant': 'Mango',
    'Pack Size (L)': 1.0,
    'Launch Date': '2023-09-01',
    'City_ID': 'CT001',
    'Date': pd.Timestamp('2023-09-10'),
    'Channel': 'E Commerce',
    'Mentions_Count': 500,
    'Sentiment_Score': 70.0,
    'Share_of_Voice': 40.0,
    'Day_of_week': 6,
    'Month': 9,
    'Is_month_end': 0
}

# Create input DataFrame
input_data = pd.DataFrame([new_product])
input_data['Days_Since_Launch'] = (input_data['Date'] - pd.to_datetime(input_data['Launch Date'])).dt.days
input_data = input_data.merge(cities, on='City_ID')
input_data = align_and_fill_weekly(input_data, market_influence)
input_data = input_data.merge(customer_agg.rename(columns={'Customer_City_ID': 'City_ID'}), on='City_ID')

# Get prediction
optimal_channel, predicted_sales = predict_optimal_channel(model, input_data[features])
print(f"Optimal Channel: {optimal_channel}, Predicted Sales: {predicted_sales[0]:.2f}")

Optimal Channel: Modern Trade, Predicted Sales: 205.54
